# Build `comments.csv` from raw Everytime text dump

This notebook converts a **raw text file** (like the one you pasted in chat) into a structured
`comments.csv` that can be used for LLM moderation policy simulation.

## Expected raw format

- Multiple posts from the 자유게시판, concatenated in one text file.
- Each post block starts with a게line that is exactly:

  `자유게시판`
유
- Then the pattern is roughly:

  - 빈 줄
  - 글쓴이 (예: `익명`)
  - 날짜/시간 + `쪽지신고` (예: `11/26 23:55쪽지신고`)
  - 원글 본문 여러 줄
  - 공감/스크랩 관련 숫자와 텍스트 (`020`, `공감`, `스크랩`)
  - 댓글 헤더 줄 (예: `익명1대댓글공감쪽지신고`, `메추리야미안해ㅠㅠ대댓글공감쪽지신고`)
  - 댓글 본문 한 줄
  - 날짜/시간 줄 (예: `11/27 10:44`)
  - 이런 패턴이 반복

## Output CSV schema

This notebook will generate a `data/comments.csv` file with:

- `sample_id` : 전체에서유유니크한 번호 (1, 2, 3, …)
- `thread_id` : 같은 자유게시판 글 묶음 ID (1, 2, 3, …)
- `role` : `"post"` or `"comment"` (원글 vs 댓글/대댓글)
- `order_in_thread` : 해당 스레드 내 순서 (0=원글, 1부터 댓글)
- `text` : 실제 내용 (줄바꿈은 공백으로 합쳐서 한 줄로 저장)

---

## 사용 방법

1. 이 노트북 파일을 `comment-policy-sim/notebooks/` 아래에 둔다.
2. `comment-policy-sim/data/raw_everytime.txt` 파일을 만들고,
   자유게시판에서 긁어온 텍스트 전체를 그대로 붙여 넣는다.
3. 이 노트북에서 셀을 위에서부터 순서대로 실행하면,
   `data/comments.csv`가 자동으로 생성된다.


In [ ]:
# !pip install pandas

from pathlib import Path
import re
import pandas as pd

#PROJECT_ROOT = Path("..").resolve()
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

RAW_PATH = DATA_DIR / "raw_everytime.txt"     # <- 여기에 raw 텍스트를 저장
CSV_PATH = DATA_DIR / "comments.csv"          # <- 이 노트북이 만들어 줄 파일

print("Project root:", PROJECT_ROOT)
print("Raw path    :", RAW_PATH)
print("Output CSV  :", CSV_PATH)

if not RAW_PATH.exists():
    print("\n[WARNING] raw_everytime.txt 파일이 아직 없습니다.")
    print(" - data/raw_everytime.txt 파일을 만들고, 자유게시판에서 복사한 텍스트를 붙여 넣으세요.")


Project root: C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu
Raw path    : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\data\raw_everytime.txt
Output CSV  : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\data\comments.csv


In [ ]:
# raw_everytime.txt 읽기
if not RAW_PATH.exists():
    raise FileNotFoundError(
        f"Raw text file not found: {RAW_PATH}\n"
        "data/raw_everytime.txt 파일을 만들고 에브리타임에서 긁어온 텍스트를 붙여 넣으세요."
    )

with open(RAW_PATH, "r", encoding="utf-8", errors="ignore") as f:
    raw_text = f.read()

lines = raw_text.splitlines()
print(f"Total lines in raw file: {len(lines)}")
print("First 20 lines preview:")
for i, line in enumerate(lines[:20], start=1):
    print(f"{i:2d}: {line}")


Total lines in raw file: 949
First 20 lines preview:
 1: 자유게시판
 2: 
 3: 익명
 4: 11/27 21:44쪽지신고
 5: 얼마나 인기없으면
 6: 글이 안올라오냐
 7: 
 8: 020
 9: 공감
10: 스크랩
11: 익명1대댓글공감쪽지신고
12: 뭐?
13: 
14: 11/27 21:49
15: 익명2대댓글공감쪽지신고
16: ?
17: 
18: 11/27 22:08
19: 
20: 자유게시판


In [ ]:
def is_date_line(s: str) -> bool:
    """날짜/시간 줄인지 간단히 판별 (예: '11/26 23:55쪽지신고', '11/25 01:251')."""
    s = s.strip()
    return bool(re.match(r"^\d{2}/\d{2}", s))


def is_stats_line(s: str) -> bool:
    """공감/스크랩 이전에 나오는 숫자만 있는 줄 (예: '020', '1110')."""
    s = s.strip()
    return bool(re.match(r"^\d+$", s))


def split_threads_by_header(lines, header_keyword="자유게시판"):
    """'자유게시판' 줄을 기준으로 스레드(게시글) 단위유 나눈다."""
    threads = []
    current = []
    for line in lines:
        if line.strip() == header_keyword:
            if current:
                threads.append(current)
                current = []
            # 헤더 줄은 버리고 내용만 모음
            continue
        current.append(line)
    if current:
        threads.append(current)
    return threads


def parse_thread(thread_lines):
    """하나의 자유게시판 글 블럭에서 원글과 댓글 텍스트를 추출한다.

    반환:
    - post_text: str or None
    - comments: list[str]
    """
    lines = list(thread_lines)
    n = len(lines)
    idx = 0

    # 앞쪽 빈 줄 제거
    while idx < n and not lines[idx].strip():
        idx += 1

    if idx >= n:
        return None, []

    # 글쓴이 (예: '익명') - 지금은 그냥 건너뛰고 사용하지 않음
    author = lines[idx].strip()
    idx += 1

    # 빈 줄 스킵
    while idx < n and not lines[idx].strip():
        idx += 1

    # 날짜/시간 줄 스킵 (예: '11/26 23:55쪽지신고')
    if idx < n and is_date_line(lines[idx]):
        idx += 1

    # --- 원글 본문 추출 ---
    body_lines = []
    while idx < n:
        s = lines[idx]
        st = s.strip()
        # 공감 숫자(020 등) 또는 공감/스크랩 영역 나오면 stop
        if is_stats_line(st) or st in ("공감", "스크랩"):
            break
        body_lines.append(st)
        idx += 1

    # 본문 줄들을 하나의 문자열로 합치기 (빈 줄은 제거)
    post_text = " ".join([b for b in body_lines if b])

    # 공감/스크랩, 빈 줄 등 건너뛰고 댓글 영역으로 이동
    while idx < n:
        st = lines[idx].strip()
        if not st or st in ("공감", "스크랩") or is_stats_line(st):
            idx += 1
            continue
        break

    comments = []

    # --- 댓글 파싱 ---
    while idx < n:
        s = lines[idx].strip()

        if not s:
            idx += 1
            continue

        # 여전히 공감/스크랩/날짜/숫자 등은 스킵
        if s in ("공감", "스크랩") or is_stats_line(s) or is_date_line(s):
            idx += 1
            continue

        # 댓글/대댓글 헤더 줄: '쪽지신고' 또는 '삭제' 포함
        if ("쪽지신고" in s) or ("삭제" in s):
            header = s
            idx += 1
            text_lines = []
            # 다음 줄들 중에서 실제 댓글 본문을 모음
            while idx < n:
                s2 = lines[idx].strip()
                # 빈 줄이면 본문 끝
                if not s2:
                    idx += 1
                    break
                # 날짜/숫자/다음 헤더가 나오면 본문 끝
                if is_date_line(s2) or is_stats_line(s2) or ("쪽지신고" in s2) or ("삭제" in s2):
                    break
                text_lines.append(s2)
                idx += 1
            comment_text = " ".join(text_lines).strip()
            if comment_text:
                comments.append(comment_text)
            continue

        # 그 외 애매한 줄은 '본문만 있는 댓글'로 처리
        comments.append(s)
        idx += 1

    return post_text, comments


In [ ]:
# 스레드(게시글) 단위로 나누기
threads = split_threads_by_header(lines)
print(f"Detected {len(threads)} threads (자유게시판 글).")

# 디버깅용으로 첫 번째 스레드 앞부분만 살펴보기
for i, t in enumerate(threads[:3], start=1):
    print(f"\n--- Thread {i} (first 15 lines) ---")
    for j, line in enumerate(t[:15], start=1):
        print(f"{j:2d}: {line}")


Detected 20 threads (자유게시판 글).

--- Thread 1 (first 15 lines) ---
 1: 
 2: 익명
 3: 11/27 21:44쪽지신고
 4: 얼마나 인기없으면
 5: 글이 안올라오냐
 6: 
 7: 020
 8: 공감
 9: 스크랩
10: 익명1대댓글공감쪽지신고
11: 뭐?
12: 
13: 11/27 21:49
14: 익명2대댓글공감쪽지신고
15: ?

--- Thread 2 (first 15 lines) ---
 1: 
 2: 익명
 3: 11/27 15:56쪽지신고
 4: 그냥 궁금해서 하는 투표
 5: AI 쓰면 안되는 시험/과제에서 솔직히 쓴 적 있다/없다
 6: 
 7: 투표
 8: 투표 기능은 모바일 앱에서 만나보실 수 있어요.
 9: 
10: 060
11: 공감
12: 스크랩
13: 익명1대댓글공감쪽지신고
14: 과제는 많이들 ai 조금이라도 써봤을듯
15: 

--- Thread 3 (first 15 lines) ---
 1: 
 2: 익명
 3: 11/27 15:29쪽지신고
 4: 12월에도 용돈줌?
 5: 13만원
 6: 
 7: 070
 8: 공감
 9: 스크랩
10: 익명1대댓글공감쪽지신고
11: ㄹㅇ 궁금
12: 
13: 11/27 15:43
14: 익명1공감쪽지신고
15: 1학기때는 안줬던것 같은데...


In [ ]:
rows = []
sample_id = 1

for thread_id, tlines in enumerate(threads, start=1):
    post_text, comments = parse_thread(tlines)

    # 원글 추가
    if post_text:
        rows.append({
            "sample_id": sample_id,
            "thread_id": thread_id,
            "role": "post",
            "order_in_thread": 0,
            "text": post_text,
        })
        sample_id += 1

    # 댓글들 추가
    for k, ctext in enumerate(comments, start=1):
        rows.append({
            "sample_id": sample_id,
            "thread_id": thread_id,
            "role": "comment",
            "order_in_thread": k,
            "text": ctext,
        })
        sample_id += 1

df = pd.DataFrame(rows)
print(f"Total rows parsed: {len(df)}")
df.head(10)


Total rows parsed: 182


,sample_id,thread_id,role,order_in_thread,text
0,1,1,post,0,얼마나 인기없으면 글이 안올라오냐
1,2,1,comment,1,뭐?
2,3,1,comment,2,?
3,4,2,post,0,그냥 궁금해서 하는 투표 AI 쓰면 안되는 시험/과제에서 솔직히 쓴 적 있다/없다 ...
4,5,2,comment,1,과제는 많이들 ai 조금이라도 써봤을듯
5,6,2,comment,2,이제는 무조건 쓰는데
6,7,2,comment,3,쓸 건 써야지.. 철도끼 써보니까 돌도끼 못 쓰겠더라
7,8,2,comment,4,시험은 안썻는데 과제에 안쓸 수기 잇나?
8,9,2,comment,5,ai는 이제와서는 고도로 발전한 계산기같은거지
9,10,2,comment,6,시험은 안쓰고 과제는 쓰지


In [ ]:
df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
print(f"Saved {len(df)} rows to {CSV_PATH}")


Saved 182 rows to C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\data\comments.csv
